# 可解釋人工智慧 (Explainable AI)

## 學習目標
- 理解模型可解釋性的重要性
- 實作 Saliency Map（梯度可視化）
- 學習 Integrated Gradients
- 實作 LIME 局部解釋方法

## 對應課程
- [李宏毅 ML 2021 - Explainable AI](https://speech.ee.ntu.edu.tw/~hylee/ml/2021-spring.php)
- [HW9: Explainable AI](https://github.com/ga642381/ML2021-Spring/tree/main/HW09)

## 可解釋 AI 概述

```
為什麼需要可解釋 AI？
├── 信任: 了解模型決策依據
├── 調試: 發現模型錯誤原因
├── 公平性: 檢測偏見
└── 法規: GDPR 要求解釋權

解釋方法分類
├── 局部解釋 (Local)
│   ├── Saliency Map
│   ├── LIME
│   └── SHAP
│
└── 全局解釋 (Global)
    ├── Feature Importance
    └── Concept Activation
```

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18, ResNet18_Weights
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用設備: {device}")

# 固定隨機種子
torch.manual_seed(42)
np.random.seed(42)

---
## Part 1: 準備工作

In [ ]:
# ========== 載入預訓練模型 ==========

# 使用預訓練的 ResNet18
model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
model = model.to(device)
model.eval()

print(f"模型: ResNet18")
print(f"參數量: {sum(p.numel() for p in model.parameters()):,}")

# ImageNet 預處理
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# ImageNet 類別名稱（簡化版）
IMAGENET_CLASSES = {
    281: 'tabby cat',
    282: 'tiger cat',
    285: 'Egyptian cat',
    243: 'bull mastiff',
    244: 'boxer',
}

In [ ]:
# ========== 建立範例圖片 ==========

def create_sample_image():
    """
    建立一個簡單的範例圖片用於演示
    """
    # 建立一個有特徵的假圖片
    img = np.zeros((224, 224, 3), dtype=np.uint8)
    
    # 添加一些形狀
    # 圓形
    for i in range(224):
        for j in range(224):
            if (i - 112)**2 + (j - 112)**2 < 50**2:
                img[i, j] = [255, 200, 100]  # 橙色圓
    
    # 矩形
    img[50:100, 150:200] = [100, 150, 255]  # 藍色矩形
    
    return Image.fromarray(img)

# 建立範例圖片
sample_image = create_sample_image()

# 預處理
input_tensor = preprocess(sample_image).unsqueeze(0).to(device)
input_tensor.requires_grad = True

# 顯示原圖
plt.figure(figsize=(4, 4))
plt.imshow(sample_image)
plt.title('Sample Image')
plt.axis('off')
plt.show()

print(f"輸入張量形狀: {input_tensor.shape}")

---
## Part 2: Saliency Map（顯著性圖）

Saliency Map 顯示輸入圖像中哪些像素對模型預測影響最大。

### 原理
$$\text{Saliency}(x) = \left| \frac{\partial y_c}{\partial x} \right|$$

其中 $y_c$ 是目標類別的分數。

In [ ]:
# ========== Vanilla Saliency Map ==========

def compute_saliency_map(model, input_tensor, target_class=None):
    """
    計算 Saliency Map
    
    步驟:
    1. 前向傳播
    2. 計算目標類別對輸入的梯度
    3. 取絕對值並在通道維度取最大值
    """
    model.eval()
    input_tensor = input_tensor.clone().detach().requires_grad_(True)
    
    # 前向傳播
    output = model(input_tensor)
    
    # 如果沒有指定目標類別，使用預測類別
    if target_class is None:
        target_class = output.argmax(dim=1).item()
    
    # 計算目標類別的分數對輸入的梯度
    model.zero_grad()
    target_score = output[0, target_class]
    target_score.backward()
    
    # 獲取梯度
    gradients = input_tensor.grad.data.abs()
    
    # 在通道維度取最大值
    saliency, _ = gradients.squeeze().max(dim=0)
    
    return saliency.cpu().numpy(), target_class

# 計算 Saliency Map
saliency, pred_class = compute_saliency_map(model, input_tensor)

# 可視化
fig, axes = plt.subplots(1, 3, figsize=(12, 4))

axes[0].imshow(sample_image)
axes[0].set_title('Original Image')
axes[0].axis('off')

axes[1].imshow(saliency, cmap='hot')
axes[1].set_title(f'Saliency Map (Class: {pred_class})')
axes[1].axis('off')

# 疊加顯示
axes[2].imshow(sample_image)
axes[2].imshow(saliency, cmap='hot', alpha=0.5)
axes[2].set_title('Overlay')
axes[2].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# ========== Smooth Gradient ==========

def smooth_grad(model, input_tensor, target_class=None, n_samples=50, noise_level=0.1):
    """
    Smooth Gradient: 通過添加噪聲平滑梯度
    
    解決 Vanilla Gradient 的噪聲問題
    """
    model.eval()
    
    # 計算噪聲標準差
    stdev = noise_level * (input_tensor.max() - input_tensor.min()).item()
    
    total_gradients = torch.zeros_like(input_tensor)
    
    for _ in range(n_samples):
        # 添加噪聲
        noisy_input = input_tensor + torch.randn_like(input_tensor) * stdev
        noisy_input.requires_grad_(True)
        
        output = model(noisy_input)
        
        if target_class is None:
            target_class = output.argmax(dim=1).item()
        
        model.zero_grad()
        output[0, target_class].backward()
        
        total_gradients += noisy_input.grad.data
    
    # 平均梯度
    avg_gradients = total_gradients / n_samples
    saliency, _ = avg_gradients.abs().squeeze().max(dim=0)
    
    return saliency.cpu().numpy()

# 計算 Smooth Gradient
smooth_saliency = smooth_grad(model, input_tensor, n_samples=30)

# 比較
fig, axes = plt.subplots(1, 3, figsize=(12, 4))

axes[0].imshow(sample_image)
axes[0].set_title('Original')
axes[0].axis('off')

axes[1].imshow(saliency, cmap='hot')
axes[1].set_title('Vanilla Gradient')
axes[1].axis('off')

axes[2].imshow(smooth_saliency, cmap='hot')
axes[2].set_title('Smooth Gradient')
axes[2].axis('off')

plt.tight_layout()
plt.show()

---
## Part 3: Integrated Gradients

Integrated Gradients 是一種更可靠的歸因方法，滿足公理性要求。

### 公式
$$\text{IG}_i(x) = (x_i - x'_i) \times \int_{\alpha=0}^{1} \frac{\partial F(x' + \alpha(x - x'))}{\partial x_i} d\alpha$$

其中 $x'$ 是 baseline（通常是黑色圖像）。

In [ ]:
# ========== Integrated Gradients ==========

def integrated_gradients(model, input_tensor, target_class=None, baseline=None, n_steps=50):
    """
    計算 Integrated Gradients
    
    Args:
        model: 模型
        input_tensor: 輸入圖像
        target_class: 目標類別
        baseline: 基準圖像（默認黑色）
        n_steps: 積分步數
    """
    model.eval()
    
    if baseline is None:
        baseline = torch.zeros_like(input_tensor)
    
    # 生成插值路徑
    scaled_inputs = [
        baseline + (float(i) / n_steps) * (input_tensor - baseline)
        for i in range(n_steps + 1)
    ]
    
    # 計算每個點的梯度
    gradients = []
    
    for scaled_input in scaled_inputs:
        scaled_input = scaled_input.clone().detach().requires_grad_(True)
        
        output = model(scaled_input)
        
        if target_class is None:
            target_class = output.argmax(dim=1).item()
        
        model.zero_grad()
        output[0, target_class].backward()
        
        gradients.append(scaled_input.grad.data.clone())
    
    # 積分（黎曼和）
    avg_gradients = torch.stack(gradients).mean(dim=0)
    
    # 乘以 (input - baseline)
    integrated_grad = (input_tensor - baseline) * avg_gradients
    
    # 在通道維度求和並取絕對值
    attribution = integrated_grad.squeeze().sum(dim=0).abs()
    
    return attribution.cpu().detach().numpy()

# 計算 Integrated Gradients
ig_attribution = integrated_gradients(model, input_tensor, n_steps=30)

# 可視化
fig, axes = plt.subplots(1, 3, figsize=(12, 4))

axes[0].imshow(sample_image)
axes[0].set_title('Original')
axes[0].axis('off')

axes[1].imshow(ig_attribution, cmap='hot')
axes[1].set_title('Integrated Gradients')
axes[1].axis('off')

axes[2].imshow(sample_image)
axes[2].imshow(ig_attribution, cmap='hot', alpha=0.5)
axes[2].set_title('Overlay')
axes[2].axis('off')

plt.tight_layout()
plt.show()

---
## Part 4: LIME (Local Interpretable Model-agnostic Explanations)

LIME 通過在局部擬合簡單模型來解釋複雜模型的預測。

In [ ]:
# ========== 簡化版 LIME ==========

from sklearn.linear_model import Ridge
from skimage.segmentation import quickshift

class SimpleLIME:
    """
    簡化版 LIME 實現
    
    步驟:
    1. 將圖像分割成超像素
    2. 隨機遮蓋超像素生成擾動樣本
    3. 用模型預測擾動樣本
    4. 擬合線性模型解釋重要性
    """
    
    def __init__(self, model, preprocess):
        self.model = model
        self.preprocess = preprocess
    
    def explain(self, image, target_class, n_samples=500, n_features=10):
        """
        解釋預測
        
        Args:
            image: PIL Image
            target_class: 目標類別
            n_samples: 擾動樣本數
            n_features: 選擇的重要特徵數
        """
        # 轉換為 numpy
        img_array = np.array(image)
        
        # 超像素分割
        segments = quickshift(img_array, kernel_size=4, max_dist=200, ratio=0.2)
        n_segments = len(np.unique(segments))
        
        # 生成擾動樣本
        perturbations = np.random.randint(0, 2, (n_samples, n_segments))
        predictions = []
        
        for pert in perturbations:
            # 根據擾動遮蓋超像素
            temp_img = img_array.copy()
            for i, include in enumerate(pert):
                if not include:
                    temp_img[segments == i] = 0  # 遮蓋為黑色
            
            # 預測
            pil_img = Image.fromarray(temp_img)
            input_tensor = self.preprocess(pil_img).unsqueeze(0).to(device)
            
            with torch.no_grad():
                output = self.model(input_tensor)
                prob = F.softmax(output, dim=1)[0, target_class].item()
            
            predictions.append(prob)
        
        predictions = np.array(predictions)
        
        # 擬合線性模型
        linear_model = Ridge(alpha=1.0)
        linear_model.fit(perturbations, predictions)
        
        # 獲取特徵重要性
        importances = linear_model.coef_
        
        # 建立解釋圖
        explanation = np.zeros_like(segments, dtype=float)
        for i, imp in enumerate(importances):
            explanation[segments == i] = imp
        
        return explanation, segments

# 使用 LIME
lime_explainer = SimpleLIME(model, preprocess)

# 獲取預測類別
with torch.no_grad():
    output = model(input_tensor)
    pred_class = output.argmax(dim=1).item()

explanation, segments = lime_explainer.explain(sample_image, pred_class, n_samples=200)

# 可視化
fig, axes = plt.subplots(1, 4, figsize=(16, 4))

axes[0].imshow(sample_image)
axes[0].set_title('Original')
axes[0].axis('off')

axes[1].imshow(segments, cmap='tab20')
axes[1].set_title('Superpixels')
axes[1].axis('off')

axes[2].imshow(explanation, cmap='RdBu_r')
axes[2].set_title('LIME Explanation')
axes[2].axis('off')

# 只顯示正向貢獻的區域
mask = explanation > 0
masked_img = np.array(sample_image).copy()
masked_img[~mask] = 0
axes[3].imshow(masked_img)
axes[3].set_title('Positive Contributions')
axes[3].axis('off')

plt.tight_layout()
plt.show()

---
## Part 5: Grad-CAM

Grad-CAM 使用卷積層的梯度生成類別激活圖。

In [ ]:
# ========== Grad-CAM ==========

class GradCAM:
    """
    Grad-CAM: Gradient-weighted Class Activation Mapping
    
    利用最後一個卷積層的梯度和特徵圖生成熱力圖
    """
    
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        
        self.gradients = None
        self.activations = None
        
        # 註冊 hooks
        target_layer.register_forward_hook(self._save_activation)
        target_layer.register_full_backward_hook(self._save_gradient)
    
    def _save_activation(self, module, input, output):
        self.activations = output.detach()
    
    def _save_gradient(self, module, grad_input, grad_output):
        self.gradients = grad_output[0].detach()
    
    def generate(self, input_tensor, target_class=None):
        """
        生成 Grad-CAM 熱力圖
        """
        self.model.eval()
        
        # 前向傳播
        output = self.model(input_tensor)
        
        if target_class is None:
            target_class = output.argmax(dim=1).item()
        
        # 反向傳播
        self.model.zero_grad()
        output[0, target_class].backward()
        
        # 計算權重（全局平均池化梯度）
        weights = self.gradients.mean(dim=(2, 3), keepdim=True)  # (1, C, 1, 1)
        
        # 加權求和
        cam = (weights * self.activations).sum(dim=1, keepdim=True)  # (1, 1, H, W)
        
        # ReLU 和正規化
        cam = F.relu(cam)
        cam = cam.squeeze().cpu().numpy()
        
        # 正規化到 [0, 1]
        cam = (cam - cam.min()) / (cam.max() - cam.min() + 1e-8)
        
        return cam, target_class

# 使用 Grad-CAM（對 ResNet 的最後一個卷積層）
target_layer = model.layer4[-1].conv2
grad_cam = GradCAM(model, target_layer)

input_tensor = preprocess(sample_image).unsqueeze(0).to(device)
input_tensor.requires_grad = True

cam, pred_class = grad_cam.generate(input_tensor)

# 調整 CAM 大小以匹配輸入圖像
cam_resized = np.array(Image.fromarray((cam * 255).astype(np.uint8)).resize((224, 224))) / 255.0

# 可視化
fig, axes = plt.subplots(1, 3, figsize=(12, 4))

axes[0].imshow(sample_image)
axes[0].set_title('Original')
axes[0].axis('off')

axes[1].imshow(cam_resized, cmap='jet')
axes[1].set_title(f'Grad-CAM (Class: {pred_class})')
axes[1].axis('off')

# 疊加
axes[2].imshow(sample_image)
axes[2].imshow(cam_resized, cmap='jet', alpha=0.5)
axes[2].set_title('Overlay')
axes[2].axis('off')

plt.tight_layout()
plt.show()

---
## Part 6: 方法比較

In [ ]:
# ========== 所有方法比較 ==========

fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# 原圖
axes[0, 0].imshow(sample_image)
axes[0, 0].set_title('Original Image', fontsize=12)
axes[0, 0].axis('off')

# Vanilla Saliency
input_tensor = preprocess(sample_image).unsqueeze(0).to(device)
input_tensor.requires_grad = True
saliency, _ = compute_saliency_map(model, input_tensor)
axes[0, 1].imshow(saliency, cmap='hot')
axes[0, 1].set_title('Vanilla Saliency', fontsize=12)
axes[0, 1].axis('off')

# Smooth Gradient
smooth_sal = smooth_grad(model, input_tensor, n_samples=20)
axes[0, 2].imshow(smooth_sal, cmap='hot')
axes[0, 2].set_title('Smooth Gradient', fontsize=12)
axes[0, 2].axis('off')

# Integrated Gradients
ig = integrated_gradients(model, input_tensor, n_steps=20)
axes[1, 0].imshow(ig, cmap='hot')
axes[1, 0].set_title('Integrated Gradients', fontsize=12)
axes[1, 0].axis('off')

# Grad-CAM
axes[1, 1].imshow(cam_resized, cmap='jet')
axes[1, 1].set_title('Grad-CAM', fontsize=12)
axes[1, 1].axis('off')

# LIME
axes[1, 2].imshow(explanation, cmap='RdBu_r')
axes[1, 2].set_title('LIME', fontsize=12)
axes[1, 2].axis('off')

plt.tight_layout()
plt.show()

---
## 🎯 總結

### 解釋方法比較

| 方法 | 類型 | 優點 | 缺點 |
|------|------|------|------|
| **Saliency Map** | 梯度 | 快速、簡單 | 噪聲多 |
| **Smooth Gradient** | 梯度 | 較平滑 | 需要多次前向 |
| **Integrated Gradients** | 梯度 | 滿足公理性 | 計算較慢 |
| **Grad-CAM** | 特徵圖 | 直觀、穩定 | 解析度低 |
| **LIME** | 擾動 | 模型無關 | 計算很慢 |

### 選擇建議

```
快速調試: Saliency Map, Grad-CAM
詳細分析: Integrated Gradients, LIME
CNN 模型: Grad-CAM
任意模型: LIME, SHAP
```

### 李宏毅 HW9 技巧

```
1. 實作 Saliency Map: 注意梯度計算和正規化
2. Smooth Gradient: 適當的噪聲程度和採樣數
3. Integrated Gradients: 使用黑色作為 baseline
4. 結果可視化: 使用適當的 colormap
```

### 下一步

前往 `robustness_security/adversarial_attack.ipynb` 學習對抗攻擊！

---
## 🏋️ 練習

### 練習 1: 比較不同解釋方法在相同預測上的一致性

實作一個函數來量化不同解釋方法之間的相似度。

In [ ]:
# ========== 練習 1: 比較解釋方法的一致性 ==========

def normalize_attribution(attr):
    """正規化歸因圖到 [0, 1]"""
    attr = attr - attr.min()
    if attr.max() > 0:
        attr = attr / attr.max()
    return attr

def compute_attribution_similarity(attr1, attr2, method='spearman'):
    """
    計算兩個歸因圖的相似度
    
    Args:
        attr1, attr2: 歸因圖 (numpy arrays)
        method: 'spearman' (排名相關) 或 'topk' (top-k 重疊率)
    """
    from scipy.stats import spearmanr
    
    # 展平
    a1 = attr1.flatten()
    a2 = attr2.flatten()
    
    if method == 'spearman':
        corr, _ = spearmanr(a1, a2)
        return corr
    elif method == 'topk':
        # Top 10% 像素重疊率
        k = int(0.1 * len(a1))
        top1 = set(np.argsort(a1)[-k:])
        top2 = set(np.argsort(a2)[-k:])
        overlap = len(top1 & top2) / k
        return overlap
    
    return 0.0

def compare_explanation_methods(model, image, preprocess):
    """
    比較所有解釋方法並計算相似度矩陣
    """
    input_tensor = preprocess(image).unsqueeze(0).to(device)
    input_tensor.requires_grad = True
    
    # 計算各方法的歸因
    attributions = {}
    
    # Vanilla Saliency
    saliency, pred_class = compute_saliency_map(model, input_tensor)
    attributions['Saliency'] = normalize_attribution(saliency)
    
    # Smooth Gradient
    smooth = smooth_grad(model, input_tensor, target_class=pred_class, n_samples=20)
    attributions['SmoothGrad'] = normalize_attribution(smooth)
    
    # Integrated Gradients
    ig = integrated_gradients(model, input_tensor, target_class=pred_class, n_steps=20)
    attributions['IntGrad'] = normalize_attribution(ig)
    
    # Grad-CAM (需要調整大小)
    target_layer = model.layer4[-1].conv2
    grad_cam_obj = GradCAM(model, target_layer)
    cam, _ = grad_cam_obj.generate(input_tensor, target_class=pred_class)
    cam_resized = np.array(Image.fromarray((cam * 255).astype(np.uint8)).resize((224, 224))) / 255.0
    attributions['GradCAM'] = normalize_attribution(cam_resized)
    
    # 計算相似度矩陣
    methods = list(attributions.keys())
    n = len(methods)
    similarity_matrix = np.zeros((n, n))
    
    for i, m1 in enumerate(methods):
        for j, m2 in enumerate(methods):
            similarity_matrix[i, j] = compute_attribution_similarity(
                attributions[m1], attributions[m2], method='spearman'
            )
    
    return attributions, similarity_matrix, methods

# 執行比較
print("練習 1: 比較解釋方法的一致性")
print("=" * 50)

attributions, sim_matrix, methods = compare_explanation_methods(model, sample_image, preprocess)

# 視覺化歸因圖
fig, axes = plt.subplots(1, 4, figsize=(14, 3))
for ax, (name, attr) in zip(axes, attributions.items()):
    ax.imshow(attr, cmap='hot')
    ax.set_title(name)
    ax.axis('off')
plt.suptitle('各方法的歸因圖（已正規化）')
plt.tight_layout()
plt.show()

# 視覺化相似度矩陣
fig, ax = plt.subplots(figsize=(6, 5))
im = ax.imshow(sim_matrix, cmap='RdYlGn', vmin=-1, vmax=1)
ax.set_xticks(range(len(methods)))
ax.set_yticks(range(len(methods)))
ax.set_xticklabels(methods, rotation=45, ha='right')
ax.set_yticklabels(methods)

# 添加數值標籤
for i in range(len(methods)):
    for j in range(len(methods)):
        ax.text(j, i, f'{sim_matrix[i,j]:.2f}', ha='center', va='center', fontsize=10)

plt.colorbar(im, label='Spearman Correlation')
plt.title('解釋方法相似度矩陣')
plt.tight_layout()
plt.show()

print("\n結論：")
print("- 基於梯度的方法（Saliency, SmoothGrad, IntGrad）通常較相似")
print("- Grad-CAM 因為低解析度，可能與其他方法相關性較低")
print("- 高相關性表示方法「認同」哪些區域重要")